<a href="https://colab.research.google.com/github/Vishwa0703/GenAI/blob/main/genai4_csv_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
# Workaround to avoid following error at notebook
# Error: NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [40]:
# Huggingface libraries to run LLM.
!pip install -q -U transformers==4.40.2
!pip install -q -U accelerate==0.30.1
!pip install -q -U bitsandbytes==0.43.1
!pip install -q -U huggingface_hub==0.23.0

#LangChain related libraries
!pip install -q -U langchain==0.1.2
!pip install -q -U langchain-community==0.0.38

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf==4.2.0

#Python framework for state-of-the-art sentence, text and image embeddings.
!pip install -q -U sentence-transformers==2.7.0

# FAISS Vector Databses specific Libraries
!pip install -q -U faiss-gpu==1.7.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-text-splitters 0.3.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.1.19 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.2 requires langsmith<0.0.84,>=0.0.83, but you have langsmith 0.1.147 which is incompatible.
langchain-text-splitters 0.3.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.1.53 which is incompatible.


In [20]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain


device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
Tesla T4


In [21]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [22]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=300,
    temperature = 0.3,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Using CSV File with LangChain CSVLoader

In [23]:
from langchain_community.document_loaders.csv_loader import CSVLoader
# Load the CSV file
loader = CSVLoader(file_path='/content/drive/MyDrive/Colab Notebooks/greenhouse_gas_inventory_data_data.csv')
data = loader.load()

# Split the documents into smaller chunks
#separator=""
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs  = text_splitter.split_documents(data)

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [24]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)


In [25]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

In [26]:
#We will run an infinite loop to ask questions to LLM and retrieve answers untill the user wants to quit
import sys
chat_history = []
query = input('Prompt: ')
# Ask question related CSV file like Which country has highest green gas?
result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

Prompt: brief  of australia data
Answer:  Australia's greenhouse gas emissions in kilotonne CO2 equivalent increased from 418,674.1 kilotonne CO2 equivalent in 1991 to 491,441.7 kilotonne CO2 equivalent in 2001. In 1991, Australia emitted 133.2 kilotonne CO2 equivalent of hydrofluorocarbons (HFCs). In 1990, Australia emitted 142.5 kilotonne CO2 equivalent of HFCs.



# Using CSV file with Pandas dataframe

In [31]:
!pip install -q -U pandas==2.0.3

In [32]:
import pandas as pd
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/greenhouse_gas_inventory_data_data.csv')

In [33]:
from langchain.schema.document import Document
text = ""
for ind in df.index:
    text += f"{df['country_or_area'][ind]}  {df['year'][ind]}  {df['value'][ind]}\n#####\n"
#Converting text to LangChain documents so that StuffDocumentsChain can understand Input
documents = Document(page_content=text, metadata={"source": "local"})

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split the documents into smaller chunks
#separators=["\n\n", "\n", " ", ""]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators="[#####]")
chunked_docs  = text_splitter.split_documents([documents])

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [35]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [36]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

In [37]:
#We will run an infinite loop to ask questions to LLM and retrieve answers untill the user wants to quit
import sys
chat_history = []
query = input('Prompt: ')
# Ask question related CSV file like Which country has highest green gas?
result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

Prompt: brief  of australia data
Answer:  Australia has a population of approximately 24 million people and is located in the southern hemisphere. The country is known for its unique wildlife, diverse landscapes, and strong economy. It is also a popular tourist destination, with attractions such as Sydney Opera House, Great Barrier Reef, and Uluru.

